In [1]:
import sys
sys.path.append("../libs/")

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import shift_join_data, print_hp
from models import build_lstm
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import HyperModel
import datetime
import keras_tuner as kt
import os

Using TensorFlow backend.


# Carga de datos

In [3]:
df = pd.read_csv('../data/wb_dataset_prep.csv')
df = df.drop('country',axis=1)
iso = df['iso'].unique()    #Codigos de paises
df = df.set_index(['iso','year'])

# Dividir Datos

In [4]:
iso_test = ['PER']
iso_train = iso[(iso != iso_test[0])]
target_col = ['rgdp_growth']
features = df.columns[(df.columns!=target_col[0])]

# Normalizar

In [5]:
df_x_train = df.loc[iso_train][features].copy()
df_y_train = df.loc[iso_train][target_col].copy()

scaler_x_train = StandardScaler()
scaler_y_train = StandardScaler()

df_x_train.iloc[:,:] = scaler_x_train.fit_transform(df_x_train)
df_y_train.iloc[:,:] = scaler_y_train.fit_transform(df_y_train)

df_x_train.iloc[:,:] = np.clip(df_x_train,-5,5)
df_y_train.iloc[:,:] = np.clip(df_y_train,-5,5)

# Optimizacion

In [6]:
class LSTMHyperModel(HyperModel):

    def __init__(self, n_features_in, n_steps_out, countries, metrics = ['mae','mse','mape'], name = None, tunable = True):
        super().__init__(name=name, tunable=tunable)
        self.n_features = n_features_in
        self.n_steps_out = n_steps_out
        self.countries = countries
        self.metrics = metrics

    def build(self, hp):
        # Parametrizamos nro de capas, nro de nodos y ratio de aprendizaje
        hp_time_steps = hp.Int('steps_in',4,10,step=1)
        hp_lstm_cells = hp.Int('lstm_cells',32,356,step=16)
        hp_lstm_dropout = hp.Float('lstm_dropout',0.1,0.5,step=0.1)
        hp_dense_layers = hp.Int('dense_layers',1,5,step=1)
        hp_dense_nodes = hp.Int('dense_nodes',32,356,step=16)
        hp_dense_dropout = hp.Float('dense_dropout',0.1,0.5,step=0.1)
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
        hp_dense_activation = hp.Choice('dense_activation', values=['relu','tanh'])

        return build_lstm(
            n_steps_in = hp_time_steps,
            n_features = self.n_features,
            n_steps_out = self.n_steps_out, 
            lstm_cells = hp_lstm_cells,
            lstm_dropout = hp_lstm_dropout,
            dense_layers = hp_dense_layers,
            dense_nodes = hp_dense_nodes,
            dense_dropout = hp_dense_dropout,
            learning_rate = hp_learning_rate,
            dense_activation = hp_dense_activation,
            metrics = self.metrics)

    def fit(self, hp, model,x,y,**kwargs):
        x_s,y_s = shift_join_data(x, y, self.countries, hp.get('steps_in'), self.n_steps_out)
        mini_batch = 32
        return model.fit(x = x_s, y = y_s, batch_size = mini_batch, **kwargs)

In [7]:
n_steps_out = 3
n_features = len(features)
ajuste_path = os.path.normpath('G:/')
fecha_hora = datetime.datetime.now().strftime('%Y%m%d_%H%M')
objective = 'val_mse'
max_epochs = 100
hp_iters = 1

In [8]:
# Condicion de parada: 50 epocas despues del menor val_loss
#rmse = RootMeanSquaredError(name='rmse')
es = EarlyStopping(monitor='val_mse', mode='min', patience=50)

In [9]:
name_prj = 'LSTM'+'_'+fecha_hora
lstm_hypermodel = LSTMHyperModel(n_features, n_steps_out, iso_train)

lstm_tuner = kt.Hyperband(
    lstm_hypermodel,
    objective = objective,
    max_epochs = max_epochs,
    hyperband_iterations = hp_iters,
    directory = ajuste_path,
    project_name = name_prj,
    overwrite=True)

lstm_tuner.search(x = df_x_train, y = df_y_train, validation_split = 0.3, epochs = max_epochs
    , verbose = 2, shuffle = False, callbacks = [es])


Trial 254 Complete [00h 01m 12s]
val_mse: 0.78281170129776

Best val_mse So Far: 0.73088538646698
Total elapsed time: 01h 03m 16s
INFO:tensorflow:Oracle triggered exit


In [10]:
# guardar mejor modelo
output_path = 'ajustes/'
best_lstm_hps = lstm_tuner.get_best_hyperparameters(num_trials = 1)[0]
print_hp(output_path+name_prj+'.txt',lstm_tuner)

lstm_model = lstm_tuner.hypermodel.build(best_lstm_hps)
lstm_model.save(output_path+name_prj+'.h5')

In [11]:
# from notifications import enviar_correo
# enviar_correo("Ajuste de Parametros Finalizado!","Se ha completado: {}".format(name_prj))